In [2]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 6.6 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import re, string, emoji, contractions
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk

In [4]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
df = pd.read_csv('/kaggle/input/deepseekreviews/dataset_labeled.csv')

In [11]:
df.head()

,content,score,sentiment
0,I like this app it's really helpful and it's e...,5,positive
1,absolute beast.,5,positive
2,not fully good once start using it 😕,1,negative
3,"all time good app , it's great piece of mind 😉",5,positive
4,amazing apps. I love using this but it's hard ...,4,positive


In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_name = "ramsrigouthamg/t5_paraphraser"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [7]:
def paraphrase(text, num_return_sequences=1):
    input_text = f"paraphrase: {text} </s>"
    encoding = tokenizer(
        input_text,
        padding='longest',
        max_length=256,
        return_tensors="pt",
        truncation=True
    ).to(device)

    outputs = model.generate(
        **encoding,
        max_length=256,
        num_beams=5,
        num_return_sequences=num_return_sequences,
        early_stopping=True
    )
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

In [12]:
neutral_df = df[df['sentiment'] == 'neutral']
negative_count = df[df['sentiment'] == 'negative'].shape[0]

In [13]:
needed_aug = negative_count - len(neutral_df)
samples = neutral_df.sample(n=needed_aug, replace=True, random_state=42)

aug_texts = []
for text in tqdm(samples['content'], desc="Augmenting neutral class"):
    try:
        aug_result = paraphrase(text, num_return_sequences=1)[0]
        aug_texts.append(aug_result)
    except:
        aug_texts.append(text)

Augmenting neutral class: 100%|██████████| 4505/4505 [36:24<00:00,  2.06it/s]  


In [14]:
aug_df = pd.DataFrame({
    'content': aug_texts,
    'sentiment': ['neutral'] * len(aug_texts)
})

In [15]:
df_augmented = pd.concat([df, aug_df], ignore_index=True)

In [19]:
df_augmented['sentiment'].value_counts()

sentiment
positive    11033
negative     6785
neutral      6785
Name: count, dtype: int64

In [21]:
df_augmented.to_csv('/kaggle/working//datasets_reviews_augmented.csv', index=False)